# Step 1:  Start with a list of task topics -- TREC session 2014 track topics - 60

In a typical search scenario, a user starts to a search session with an initial query with a specific information need in-mind or not and continues to reformulate and refine their queries until their information need is satisfied or completely abandon the search.

In order to simulate the queries in a search session we use predefined information needs and user’s session history as basis for query generation.

We use TREC Session 2014 Track's task topics (https://trec.nist.gov/data/session2014.html) as predefined information needs, to simulate queries. Please download the topics from TREC Session Track repository.

Input File: file_1_session2014_topic.csv. A tab-separated file with two columns - TREC Session 2014 topic ids, and topic descriptions.

In [1]:
import os
import pandas as pd
import numpy as np

In [ ]:
topic = pd.read_csv("input/file_1_session2014_topic.csv", sep = '\t', encoding = 'latin')

In [ ]:
topic.shape
# (60,2)

In [ ]:
topic.columns
#Index(['topic_id', 'session_topic'], dtype='object')

# Step 2: Generate first query

Automatically simulating the first query is always difficult as there is no prior search history or user activities to consider for a new information need.

To keep things simple, we use a two-phase process by which we first generate sample queries based on the task/topic description by sampling from a language model called KeyBERT, then assign scores to the generated queries based on their similarity with the topic descriptions. We use KeyBERT, a basic and easy-to-use keyword extraction technique. KeyBERT generates the most probable terms and keyphrases for a query given a task description using BERT. The model uses BERT-embeddings and cosine similarity to locate similar words and keyphrases in a document or large text description that are the most similar to the document itself. The most similar terms are then identified as the ones that best describe the entire document, here the information need. KeyBERT generates embeddings using huggingface transformer-based pre-trained models.

In [ ]:
query_gen = topic.copy()
query_gen['bert'] = 0

In [ ]:
!pip install keybert

In [ ]:
from keybert import KeyBERT

In [ ]:
kw_model = KeyBERT()

In [ ]:
for i in topic.index:
     text = topic['session_topic'][i]
     keywords = kw_model.extract_keywords(text)
     query_gen['bert'][i] = keywords

In [ ]:
FirstQuery = query_gen
FirstQuery.head(5)

In [ ]:
import re

In [ ]:
FirstQuery['query'] = 0

In [ ]:
for i in FirstQuery.index:
     text = FirstQuery['bert'][i]
     text2 = str(text).replace('[(\'', '')
     text2 = re.sub(r',( [0-9]+\.[0-9]+)\)\, \(', '', str(text2))
     text2 = re.sub(r',( [0-9]+\.[0-9]+)\)', '', str(text2))
     text2 = str(text2).replace('\']', '')
     text2 = str(text2).replace('\'\'', ' ')
     FirstQuery['query'][i] = text2

In [ ]:
FirstQuery['query_id'] = 0

In [ ]:
FirstQuery = FirstQuery.drop('bert', axis = 1)

In [ ]:
FirstQuery.reset_index(inplace=True)
FirstQuery = FirstQuery.rename(columns = {'index':'session_id'})

In [ ]:
FirstQuery.to_csv("input/file_2_firstQuery.csv", sep = '\t', index = None)

# Step 3: Develop first query segment - for each query retrieve a set of ranked docuemnts

The documents can be collected based on the search context and corpus.

Note: For the sake of reproducibility, we used the Google API to retrieve top-10 relevant documents for each query; however, we also recommend ClueWeb 2012 corpus (http://lemurproject.org/clueweb12/) for TREC Session Track topics.

Also, we only extracted docuement URLs and titles.

In [ ]:
!pip install beautifulsoup4
!pip install google

In [ ]:
results = pd.read_csv("input/file_2_firstQuery.csv", sep = '\t', encoding = 'latin')

In [ ]:
try:
    from googlesearch import search
except ImportError:
    print("No module found")
 
large = []
for i in results.index:
     query = results['query'][i]
     result = []
     for j in search(query, tld="com", num=10, stop=10, pause=2):
       result.append(j)
     large.extend(result)

In [ ]:
df = pd.DataFrame(large, columns = ['doc'])
df.shape

In [ ]:
df['id'] = range(0, len(df))

In [ ]:
df['session_id'] = df['id'] // 10

In [ ]:
df['session_id'].value_counts()

In [ ]:
query_result = pd.merge(results, df, on = 'session_id', how = 'outer')

In [ ]:
query_result = query_result.drop('session_topic', axis =1)
query_result = query_result.rename(columns={'doc': 'doc_url'})

In [ ]:
query_result.to_csv('input/file_3_first_segment.csv', sep = '\t', index = None)

In [ ]:
query_result = pd.read_csv("input/file_3_first_segment.csv", sep = "\t", encoding = 'latin')

In [ ]:
!pip install requests
!pip install bs4

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
large = []
for i in query_result.index:
     url = query_result['doc_url'][i]
     reqs = requests.get(url)
     soup = BeautifulSoup(reqs.text, 'html.parser')
     for j in soup.find_all('title'):
       large.append(j.get_text())

In [ ]:
df = pd.DataFrame(large, columns = ['title'])

In [ ]:
result_title = pd.merge(query_result, df, how = 'left', left_index=True, right_index=True)

In [ ]:
result_title.to_csv('input/file_4_first_segment.csv', sep = '\t', index = None)

# Step 4: Extract LETOR features for each query-document pair

For each query-document pair, we extract several features. Each row of the dataset represents a feature vector for a query-document pair.

For the demo purposes here, when extracting the features, we tried to extract some of the standard features in LETOR (https://arxiv.org/ftp/arxiv/papers/1306/1306.2597.pdf), tried to reproduce additional query and session-level features. We can extract many low-level and textual features based on the original features.

In [ ]:
import operator
import pickle
import nltk 
import re
import string
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import operator
import math
from __future__ import division

In [ ]:
nltk.download('punkt')

In [ ]:
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

In [ ]:
!pip install -U spacy==3.0.0

In [ ]:
import spacy
!python -m spacy download en_core_web_sm

In [ ]:
!pip install strsim

In [ ]:
!pip install texthero #-U

In [ ]:
from sklearn import preprocessing
from similarity.jarowinkler import JaroWinkler
from similarity.cosine import Cosine

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
stop_words = set(stopwords.words('english')) 

In [ ]:
data = pd.read_csv("input/file_4_first_segment.csv", sep = "\t")

In [ ]:
data['clean_url'] = data.doc_url.replace(to_replace='https://',value='',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='www.',value='',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='.com',value='',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='.html',value='',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='/',value=' ',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='-',value=' ',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='  ',value='',regex=True)
data['clean_url'] = data['clean_url'].apply(lambda x:x.strip())
data["clean_url"] = data["clean_url"].str.replace(r'http', '', regex=True).replace(r'www.', '', regex=True).replace(r'com', '', regex=True)

data['clean_query'] = data['query'].replace(to_replace='/',value=' ',regex=True)
data['clean_query'] = data['clean_query'].replace(to_replace='-',value=' ',regex=True)
data['clean_query'] = data['clean_query'].replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
data['clean_query'] = data['clean_query'].replace(to_replace='  ',value='',regex=True)
data['clean_query'] = data['clean_query'].apply(lambda x:x.strip())

data['clean_title'] = data['title'].replace(to_replace='/',value=' ',regex=True)
data['clean_title'] = data['clean_title'].replace(to_replace='-',value=' ',regex=True)
data['clean_title'] = data['clean_title'].replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
data['clean_title'] = data['clean_title'].replace(to_replace='  ',value='',regex=True)
data['clean_title'] = data['clean_title'].apply(lambda x:x.strip())

In [ ]:
data['clean_query']=[entry.lower() for entry in data['clean_query']]
data['clean_url']=[entry.lower() for entry in data['clean_url']]
data['clean_title']=[entry.lower() for entry in data['clean_title']]

In [ ]:
data['query_length'] = data['clean_query'].str.len()

In [ ]:
data['url_length'] = data['clean_url'].str.len()

In [ ]:
data['title_length'] = data['clean_title'].str.len()

In [ ]:
data['tf_url'] = 0
data['tf_url'] = data['tf_url'].astype('float')

#data['tf_title'] = 0
#data['tf_title'] = data['tf_title'].astype('float')

for i in data.index:
  for j in data.query_id:
    query = data['clean_query'][i]
    tokens1 = word_tokenize(query)
    list1 = []
    for token1 in tokens1:
      if token1 not in stop_words:
        lemma1 = wordnet_lemmatizer.lemmatize(token1)
        list1.append(lemma1)
    bagOfWordsQuery = list1

    url = data['clean_url'][i] #clean_title
    tokens2 = word_tokenize(url)
    list2 = []
    for token2 in tokens2:
      if token2 not in stop_words:
        lemma2 = wordnet_lemmatizer.lemmatize(token2)
        list2.append(lemma2)
    bagOfWordsurl = list2

    uniqueWords = set(bagOfWordsQuery).union(set(bagOfWordsurl))
    
    numOfWordsQuery = dict.fromkeys(uniqueWords, 0)
    for word in bagOfWordsQuery:
      numOfWordsQuery[word] += 1
    numOfWordsurl = dict.fromkeys(uniqueWords, 0)
    for word in bagOfWordsurl:
      numOfWordsurl[word] += 1

    tfDict = {}
    bagOfWordsCount = len(bagOfWordsurl)
    for word, count in numOfWordsurl.items():
      if word in bagOfWordsQuery:
        if bagOfWordsCount == 0:
          tfDict[word] = 0.0
        else:
          tfDict[word] = np.float(count) / np.float(bagOfWordsCount)
      else:
        tfDict[word] = 0.0
    tf = tfDict
  values = tf.values()
  data['tf_url'][i] = sum(values) #tf_title

In [ ]:
data2 = data.copy()

In [ ]:
data2.set_index(['clean_query'], inplace=True)

In [ ]:
data2.sort_values(by=['query'], inplace=True)

In [ ]:
df = data[['clean_query', 'clean_url', 'clean_title']]

In [ ]:
df2 = df.groupby('clean_query', as_index = False).agg(lambda x: x.tolist())
df2 = df2.rename(columns={'clean_url': 'list_doc', 'clean_title': 'list_title'})
df2.set_index(['clean_query'], inplace=True)

In [ ]:
df3 = data2.join(df2).reset_index()

In [ ]:
df3.sort_values(by=['topic_id'], inplace=True)

In [ ]:
data = df3.copy()

In [ ]:
data['list_doc'] = data['list_doc'].astype(str).str.strip('[|]')
data['list_doc'] = data['list_doc'].str.replace(r"[\"\',]", '')
data['list_doc'] = data['list_doc'].replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
data['list_doc'] = data['list_doc'].replace(to_replace='  ',value=' ',regex=True)

data['list_title'] = data['list_title'].astype(str).str.strip('[|]')
data['list_title'] = data['list_title'].str.replace(r"[\"\',]", '')
data['list_title'] = data['list_title'].replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
data['list_title'] = data['list_title'].replace(to_replace='  ',value=' ',regex=True)

In [ ]:
#data['url_idf'] = 0
#data['url_idf'] = data['url_idf'].astype('float')

data['title_idf'] = 0
data['title_idf'] = data['title_idf'].astype('float')

for i in data.index:
  for j in data.query_id:
    query = data['clean_query'][i]
    tokens1 = word_tokenize(query)
    list1 = []
    for token1 in tokens1:
      if token1 not in stop_words:
        lemma1 = wordnet_lemmatizer.lemmatize(token1)
        list1.append(lemma1)
    bagOfWordsQuery = list1

    url = data['list_title'][i] #list_doc
    tokens2 = word_tokenize(url)
    list2 = []
    for token2 in tokens2:
      if token2 not in stop_words:
        lemma2 = wordnet_lemmatizer.lemmatize(token2)
        list2.append(lemma2)
    bagOfWordsurl = list2

    uniqueWords = set(bagOfWordsQuery).union(set(bagOfWordsurl))
    numOfWordsQ = dict.fromkeys(uniqueWords, 0)
    for word in bagOfWordsQuery:
      numOfWordsQ[word] += 1
    numOfWordsurl = dict.fromkeys(uniqueWords, 0)
    for word in bagOfWordsurl:
      numOfWordsurl[word] += 1

    documents = [numOfWordsQ, numOfWordsurl]
    N = len(documents)
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
      for word, val in document.items():
        if val > 0:
          idfDict[word] += 1
    
    for word, val in idfDict.items():
      idfDict[word] = math.log(N / float(val))
    idfs = idfDict
  values = idfs.values()
  data['title_idf'][i] = sum(values) #url_idf

In [ ]:
data['tfidf_url'] = 0
data['tfidf_url'] = data['tfidf_url'].astype('float')

data['tfidf_title'] = 0
data['tfidf_title'] = data['tfidf_title'].astype('float')

data['tfidf_url'] = data['tf_url'] * data['url_idf']
data['tfidf_title'] = data['tf_title'] * data['title_idf']

In [ ]:
!pip install rank_bm25

In [ ]:
from nltk.stem import PorterStemmer
from rank_bm25 import *

In [ ]:
ps = PorterStemmer()

In [ ]:
data['url_bm25'] = 0
data['url_bm25'] = data['url_bm25'].astype('float')
data['title_bm25'] = 0
data['title_bm25'] = data['title_bm25'].astype('float')

In [ ]:
for i in data.index:
  for j in data.query_id:
    doc = []
    query = data['clean_query'][i]
    tokens1 = word_tokenize(query)
    list1 = []
    for token1 in tokens1:
      if token1 not in stop_words:
        stem1 = ps.stem(token1)
        list1.append(stem1)
    bagOfWordsQuery = list1

    url = data['list_title'][i] #list_doc
  doc.append(url)

  tokens2 = word_tokenize(url)
  list2 = []
  for token2 in tokens2:
    if token2 not in stop_words:
      stem2 = ps.stem(token2)
      list2.append(stem2)
  bagOfWordsU = list2
  bm25 = BM25Okapi(bagOfWordsU)

  doc_scores = bm25.get_scores(bagOfWordsQuery)
  data['title_bm25'][i] = np.sum(doc_scores) #url_bm25

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from similarity.jarowinkler import JaroWinkler

from similarity.cosine import Cosine

In [ ]:
jarowinkler = JaroWinkler()
data["url_jarowinkler_sim"] = [jarowinkler.similarity(i,j) for i,j in zip(data["clean_query"],data["clean_url"])]
data["title_jarowinkler_sim"] = [jarowinkler.similarity(i,j) for i,j in zip(data["clean_query"],data["clean_title"])]

In [ ]:
def jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
data['url_jac_sim'] = [jaccard_sim(text1, text2) for text1, text2 in zip(data['clean_query'], data['clean_url'])]
data['title_jac_sim'] = [jaccard_sim(text1, text2) for text1, text2 in zip(data['clean_query'], data['clean_title'])]

In [ ]:
data = data.sort_values('id').reset_index()

In [ ]:
data['doc_rank'] = data.groupby("query")['clean_url'].transform(lambda x: x.ne(x.shift()).cumsum())

In [ ]:
data = data.drop(['level_0', 'index'], axis = 1)

In [ ]:
data.to_csv("input/file_4_first_qsegment_all_features.csv", sep = '\t', index=False, header=True)

In [ ]:
df.to_csv("input/file_5_first_qsegment_all_features.csv", sep = '\t', index=False, header=True)

# Step 5: Click modeling

Generate users' clicks on documents using a click model based on Gassian Mixture Model

In [475]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.cluster import KMeans
import time
from collections import Counter
from sklearn.mixture import GaussianMixture
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,LabelBinarizer, MultiLabelBinarizer, LabelEncoder, OrdinalEncoder

In [480]:
data = pd.read_csv("input/file_5_first_qsegment_all_features.csv", sep = '\t')

In [481]:
data['doc_rank'] = data['doc_rank'].astype('category')
data['TopicProduct'] = data['TopicProduct'].astype('category')
data['TopicGoal'] = data['TopicGoal'].astype('category')
data['TaskType'] = data['TaskType'].astype('category')

In [482]:
numeric_columns = ['query_length', 'url_length',
       'title_length', 'tf_title', 'tf_url',
       'url_idf', 'title_idf', 'tfidf_url', 'tfidf_title', 'url_bm25',
       'title_bm25', 'url_jarowinkler_sim',
       'title_jarowinkler_sim', 'url_jac_sim', 'title_jac_sim']
categorical_columns = ['TopicProduct', 'TopicGoal', 'TaskType']
ordinal_columns = ['doc_rank']

In [483]:
data[numeric_columns] = StandardScaler().fit_transform(data[numeric_columns])

In [484]:
data = pd.get_dummies(data, columns=['TopicProduct', 'TopicGoal', 'TaskType'], drop_first=True)

In [485]:
data[ordinal_columns] = OrdinalEncoder().fit_transform(data[ordinal_columns])

In [487]:
X = data.drop(['session_id', 'query', 'query_id', 'doc_url', 'title', 'clean_url', 'clean_title', 'list_doc', 'list_title'], axis = 1)

In [488]:
X = X.set_index('clean_query')

In [489]:
gmm = GaussianMixture(n_components=2, covariance_type="full", random_state=0)

In [490]:
list_of_df = list()

for i in X.index:
  df = X[X.index==i]
  gmm.fit(df)
  labels = gmm.predict(df)
  df['click']= labels
  list_of_df.append(df)

master = pd.concat(list_of_df, ignore_index=False)

In [491]:
df1 = master.drop_duplicates( keep='last')
#df2 = master.drop_duplicates(keep='first')

In [500]:
data = data[['id', 'session_id', 'query', 'query_id', 'doc_url', 'title', 'clean_url', 'clean_title', 'list_doc', 'list_title']]

In [501]:
data = data.sort_values('id').reset_index()

In [502]:
df1 = df1.sort_values('id').reset_index()

In [503]:
data2 = pd.merge(df1, data, on = 'id')

In [507]:
data2 = data2.drop(['index_x', 'index_y'], axis =1)

In [ ]:
data2.shape

In [509]:
data2.to_csv("input/file_6_first_qsegment_click_features.csv", sep = '\t', index=False, header=True)

# Step 6: Compute session, query level temporal features

We treat temporal features prediction as a regression problem. We build a random frest regression model from scratch, trained the model with similar features from TREC Session Track 2014 (as the dataset contains temporal information), saved the pre-trained model, and later loaded here to predict user dwell time on query segments.

In [10]:
from __future__ import division
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
sc = StandardScaler()

In [99]:
data = pd.read_csv('input/file_6_first_qsegment_click_features.csv', sep = '\t')

In [81]:
prep = data.drop(['clean_query', 'session_id', 'query', 'query_id',
       'doc_url', 'title', 'clean_url', 'clean_title', 'list_doc',
       'list_title'], axis =1)
prep = prep.set_index("topic_id")

In [86]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)

In [91]:
with open('pre_trained_models/serp_rf_model_pkl' , 'rb') as f:
   serp_rf_pretrained = pickle.load(f)

In [92]:
with open('pre_trained_models/query_rf_model_pkl' , 'rb') as f:
   query_rf_pretrained = pickle.load(f)

In [93]:
with open('pre_trained_models/refor_rf_model_pkl' , 'rb') as f:
   refor_rf_pretrained = pickle.load(f)

In [94]:
test = prep.copy()

In [96]:
  test['SERPDwelltime'] = 0
  test['QueryDwelltime'] = 0
  test['SERPDwelltime']= test['SERPDwelltime'].astype('float')
  test['QueryDwelltime']= test['QueryDwelltime'].astype('float')

In [98]:
y_pred_serp = []
y_pred_query = []
#y_pred_refor = []

for i in test.index:
  X = test[test.index==i]
  X_prep_serp = X.drop(["SERPtime", 'SERPDwelltime', 'QueryDwelltime'], axis=1)
  X_prep_serp  = X_prep_serp.to_numpy()
  X_prep_query = X.drop(["QueryDwellTime", 'QueryDwelltime', 'SERPDwelltime'], axis=1).to_numpy()
  #X_prep_refor = X.drop(["ReformulationTime"], axis=1).to_numpy()
  serp = serp_rf_pretrained.predict(X_prep_serp)
  query = query_rf_pretrained.predict(X_prep_query)
  #ypred_refor = refor_rf_pretrained.predict(X_prep_refor)
  y_pred_serp.append(serp)
  y_pred_query.append(query)

In [101]:
data['SERPDwelltime'] = serp_rf_pretrained.predict(prep_serp)
data['QueryDwelltime'] = query_rf_pretrained.predict(prep_query)

In [105]:
data.to_csv("input/file_7_first_qsegment_all_features.csv", sep = '\t', index=False, header=True)

# Step 7: Query reformulation Behavior

Generate query reformulation for each task session. To reformulate queries, we took into account the previous queries and the documents users clicked in previous query segments.

We then applied a BERT language model to generate possible query sentences.

In ideal search sessions, users would reformulate queries and reviewed retrieved results until they satisfied with the information. For the demo dataset purposes, we only reformulate queries with possible combinations.

In [3]:
data = pd.read_csv('input/file_7_first_qsegment_all_features.csv', sep = '\t')

In [4]:
data.sort_values(by=['id'], inplace=True)

In [70]:
prep = data.loc[data['click'] ==1]

In [ ]:
!pip install keybert

In [25]:
from keybert import KeyBERT

In [ ]:
kw_model = KeyBERT()

In [ ]:
list_of_df = []
for i in prep.index:
  text = prep['clean_query'][i] + prep ['clean_url'][i]
  #print(text)
  keywords = kw_model.extract_keywords(text)
  print(keywords)
  list_of_df.append(keywords)

In [60]:
df = pd.DataFrame({'new_query':list_of_df})

In [ ]:
se = pd.Series(list_of_df)
prep['new_query'] = se.values

In [84]:
prep = prep[['id', 'topic_id', 'session_id', 'new_query', 'TopicProduct_1', 'TopicGoal_1', 'TaskType_1',
       'TaskType_2', 'TaskType_3']]

In [89]:
import re

In [ ]:
prep['query'] = 0
for i in prep.index:
     text = prep['new_query'][i]
     text2 = str(text).replace('[(\'', '')
     text2 = re.sub(r',( [0-9]+\.[0-9]+)\)\, \(', '', str(text2))
     text2 = re.sub(r',( [0-9]+\.[0-9]+)\)', '', str(text2))
     text2 = str(text2).replace('\']', '')
     text2 = str(text2).replace('\'\'', ' ')
     prep['query'][i] = text2

In [108]:
prep.sort_values(by=['id', 'topic_id'], inplace=True)

In [109]:
prep['query_id'] = prep.groupby('topic_id').cumcount()+1 #.transform(lambda x: x.ne(x.shift()).count())

In [111]:
prep = prep.drop('new_query', axis =1)

In [ ]:
prep['topic_id'].value_counts()

In [ ]:
!pip install beautifulsoup4
!pip install google

In [118]:
import tensorflow as tf
%tensorflow_version 2.x

In [119]:
try:
    from googlesearch import search
except ImportError:
    print("No module named found")

large = []
for i in prep.index:
     query = prep['query'][i]
     result = []
     for j in search(query, tld="com", num=10, stop=10, pause=2):
       result.append(j)
     large.extend(result)

In [ ]:
df = pd.DataFrame(large, columns = ['doc_url'])

In [142]:
df['id'] = range(0, len(df))

In [143]:
dd = prep.copy()

In [144]:
dd = prep[['topic_id']]

In [146]:
dd1 =dd.loc[dd.index.repeat(10)]

In [152]:
dd1['id'] = range(0, len(dd1))

In [154]:
dd1.reset_index(inplace=True)
dd1 = dd1.rename(columns = {'index':'or_id'})

In [156]:
result = pd.concat([df, dd1], axis=1)

In [161]:
result = result.dropna()

In [166]:
query_result = pd.merge(result, prep, left_on = ['or_id', 'topic_id'], right_on = ['id','topic_id'], how = 'outer')

In [169]:
query_result = query_result.drop(['id_x', 'id_x', 'or_id'], axis = 1)

In [172]:
query_result.to_csv('input/file_8_qsegment_refor.csv', sep = '\t')

# Step 8: Extract LETOR features session level

In [175]:
data = pd.read_csv('input/file_8_qsegment_refor.csv', sep = ',')

In [ ]:
data.shape

In [177]:
import operator
import pickle
import nltk 
import re
import string
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer

In [178]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import operator
import math
from __future__ import division

In [ ]:
nltk.download('punkt')

In [ ]:
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

In [ ]:
!pip install -U spacy==3.0.0

In [ ]:
import spacy
!python -m spacy download en_core_web_sm

In [ ]:
!pip install strsim

In [ ]:
!pip install texthero #-U

In [184]:
from sklearn import preprocessing
from similarity.jarowinkler import JaroWinkler
from similarity.cosine import Cosine

In [185]:
wordnet_lemmatizer = WordNetLemmatizer()

In [186]:
stop_words = set(stopwords.words('english')) 

In [188]:
data['clean_url'] = data.doc_url.replace(to_replace='https://',value='',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='www.',value='',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='.com',value='',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='.html',value='',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='/',value=' ',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='-',value=' ',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
data['clean_url'] = data['clean_url'].replace(to_replace='  ',value='',regex=True)
data['clean_url'] = data['clean_url'].apply(lambda x:x.strip())
data["clean_url"] = data["clean_url"].str.replace(r'http', '', regex=True).replace(r'www.', '', regex=True).replace(r'com', '', regex=True)

data['clean_query'] = data['query'].replace(to_replace='/',value=' ',regex=True)
data['clean_query'] = data['clean_query'].replace(to_replace='-',value=' ',regex=True)
data['clean_query'] = data['clean_query'].replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
data['clean_query'] = data['clean_query'].replace(to_replace='  ',value='',regex=True)
data['clean_query'] = data['clean_query'].apply(lambda x:x.strip())

In [189]:
data['clean_query']=[entry.lower() for entry in data['clean_query']]
data['clean_url']=[entry.lower() for entry in data['clean_url']]

In [190]:
data['query_length'] = data['clean_query'].str.len()

In [191]:
data['url_length'] = data['clean_url'].str.len()

In [ ]:
data['tf_url'] = 0
data['tf_url'] = data['tf_url'].astype('float')

for i in data.index:
  for j in data.query_id:
    query = data['clean_query'][i]
    tokens1 = word_tokenize(query)
    list1 = []
    for token1 in tokens1:
      if token1 not in stop_words:
        lemma1 = wordnet_lemmatizer.lemmatize(token1)
        list1.append(lemma1)
    bagOfWordsQuery = list1

    url = data['clean_url'][i]
    tokens2 = word_tokenize(url)
    list2 = []
    for token2 in tokens2:
      if token2 not in stop_words:
        lemma2 = wordnet_lemmatizer.lemmatize(token2)
        list2.append(lemma2)
    bagOfWordsurl = list2

    uniqueWords = set(bagOfWordsQuery).union(set(bagOfWordsurl))
    
    numOfWordsQuery = dict.fromkeys(uniqueWords, 0)
    for word in bagOfWordsQuery:
      numOfWordsQuery[word] += 1
    numOfWordsurl = dict.fromkeys(uniqueWords, 0)
    for word in bagOfWordsurl:
      numOfWordsurl[word] += 1

    tfDict = {}
    bagOfWordsCount = len(bagOfWordsurl)
    for word, count in numOfWordsurl.items():
      if word in bagOfWordsQuery:
        if bagOfWordsCount == 0:
          tfDict[word] = 0.0
        else:
          tfDict[word] = np.float(count) / np.float(bagOfWordsCount)
      else:
        tfDict[word] = 0.0
    tf = tfDict
  values = tf.values()
  data['tf_url'][i] = sum(values)

In [193]:
data2 = data.copy()

In [194]:
data2.set_index(['clean_query'], inplace=True)

In [196]:
data2.sort_values(by=['query'], inplace=True)

In [198]:
df = data[['clean_query', 'clean_url']]

In [199]:
df2 = df.groupby('clean_query', as_index = False).agg(lambda x: x.tolist())
df2 = df2.rename(columns={'clean_url': 'list_doc'})
df2.set_index(['clean_query'], inplace=True)

In [200]:
df3 = data2.join(df2).reset_index()

In [201]:
df3.sort_values(by=['topic_id'], inplace=True)

In [203]:
data = df3.copy()

In [204]:
data['list_doc'] = data['list_doc'].astype(str).str.strip('[|]')
data['list_doc'] = data['list_doc'].str.replace(r"[\"\',]", '')
data['list_doc'] = data['list_doc'].replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
data['list_doc'] = data['list_doc'].replace(to_replace='  ',value=' ',regex=True)

In [ ]:
data['url_idf'] = 0
data['url_idf'] = data['url_idf'].astype('float')

for i in data.index:
  for j in data.query_id:
    query = data['clean_query'][i]
    tokens1 = word_tokenize(query)
    list1 = []
    for token1 in tokens1:
      if token1 not in stop_words:
        lemma1 = wordnet_lemmatizer.lemmatize(token1)
        list1.append(lemma1)
    bagOfWordsQuery = list1

    url = data['list_doc'][i] #list_doc
    tokens2 = word_tokenize(url)
    list2 = []
    for token2 in tokens2:
      if token2 not in stop_words:
        lemma2 = wordnet_lemmatizer.lemmatize(token2)
        list2.append(lemma2)
    bagOfWordsurl = list2

    uniqueWords = set(bagOfWordsQuery).union(set(bagOfWordsurl))
    numOfWordsQ = dict.fromkeys(uniqueWords, 0)
    for word in bagOfWordsQuery:
      numOfWordsQ[word] += 1
    numOfWordsurl = dict.fromkeys(uniqueWords, 0)
    for word in bagOfWordsurl:
      numOfWordsurl[word] += 1

    documents = [numOfWordsQ, numOfWordsurl]
    N = len(documents)
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
      for word, val in document.items():
        if val > 0:
          idfDict[word] += 1
    
    for word, val in idfDict.items():
      idfDict[word] = math.log(N / float(val))
    idfs = idfDict
  values = idfs.values()
  data['url_idf'][i] = sum(values) #url_idf

In [208]:
data['tfidf_url'] = 0
data['tfidf_url'] = data['tfidf_url'].astype('float')

data['tfidf_url'] = data['tf_url'] * data['url_idf']

In [ ]:
!pip install rank_bm25

In [211]:
from nltk.stem import PorterStemmer
from rank_bm25 import *

In [212]:
ps = PorterStemmer()

In [213]:
data['url_bm25'] = 0
data['url_bm25'] = data['url_bm25'].astype('float')

In [ ]:
for i in data.index:
  for j in data.query_id:
    doc = []
    query = data['clean_query'][i]
    tokens1 = word_tokenize(query)
    list1 = []
    for token1 in tokens1:
      if token1 not in stop_words:
        stem1 = ps.stem(token1)
        list1.append(stem1)
    bagOfWordsQuery = list1

    url = data['list_doc'][i] #list_doc
  doc.append(url)

  tokens2 = word_tokenize(url)
  list2 = []
  for token2 in tokens2:
    if token2 not in stop_words:
      stem2 = ps.stem(token2)
      list2.append(stem2)
  bagOfWordsU = list2
  bm25 = BM25Okapi(bagOfWordsU)

  doc_scores = bm25.get_scores(bagOfWordsQuery)
  data['url_bm25'][i] = np.sum(doc_scores) #url_bm25

In [215]:
from sklearn.metrics.pairwise import cosine_similarity
from similarity.jarowinkler import JaroWinkler

from similarity.cosine import Cosine

In [216]:
jarowinkler = JaroWinkler()
data["url_jarowinkler_sim"] = [jarowinkler.similarity(i,j) for i,j in zip(data["clean_query"],data["clean_url"])]

In [217]:
def jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [218]:
data['url_jac_sim'] = [jaccard_sim(text1, text2) for text1, text2 in zip(data['clean_query'], data['clean_url'])]

In [220]:
data = data.sort_values('id').reset_index()

In [222]:
data['doc_rank'] = data.groupby("query")['clean_url'].transform(lambda x: x.ne(x.shift()).cumsum())

In [223]:
data = data.drop(['index'], axis = 1)

In [226]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.cluster import KMeans
import time
from collections import Counter
from sklearn.mixture import GaussianMixture
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,LabelBinarizer, MultiLabelBinarizer, LabelEncoder, OrdinalEncoder

In [228]:
data['doc_rank'] = data['doc_rank'].astype('category')

In [229]:
numeric_columns = ['query_length', 'url_length', 'tf_url',
       'url_idf', 'tfidf_url', 'url_bm25',
       'url_jarowinkler_sim', 'url_jac_sim']
ordinal_columns = ['doc_rank']

In [230]:
data[numeric_columns] = StandardScaler().fit_transform(data[numeric_columns])

In [231]:
data[ordinal_columns] = OrdinalEncoder().fit_transform(data[ordinal_columns])

In [301]:
X = data.drop(['clean_query', 'session_id', 'query_id', 'doc_url', 'clean_url', 'list_doc', 'query'], axis = 1)

In [302]:
X = X.set_index('id')

In [304]:
gmm = GaussianMixture(n_components=2, covariance_type="full", random_state=0)

In [ ]:
gmm.fit(X)

In [307]:
data['click'] = gmm.predict(X)

In [308]:
data = data.sort_values('id').reset_index()

In [313]:
from __future__ import division
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
sc = StandardScaler()

In [323]:
prep = data.drop(['clean_query', 'session_id', 'query', 'query_id',
       'doc_url', 'clean_url', 'list_doc'], axis =1)
prep = prep.set_index("topic_id")

In [ ]:
prep_serp = prep.drop(["SERPtime"], axis=1).to_numpy()
prep_query = prep.drop(["QueryDwellTime"], axis=1).to_numpy()
prep_query = prep.drop(["ReformulationTime"], axis=1).to_numpy()

In [316]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)

In [317]:
with open('pre_trained_models/serp2_rf_model_pkl' , 'rb') as f:
   serp_rf_pretrained = pickle.load(f)

In [318]:
with open('pre_trained_models/query2_rf_model_pkl' , 'rb') as f:
   query_rf_pretrained = pickle.load(f)

In [319]:
with open('pre_trained_models/refor2_rf_model_pkl' , 'rb') as f:
   refor_rf_pretrained = pickle.load(f)

In [320]:
test = prep.copy()

In [321]:
  test['SERPDwelltime'] = 0
  test['QueryDwelltime'] = 0
  test['Reformulationtime'] = 0
  test['SERPDwelltime']= test['SERPDwelltime'].astype('float')
  test['QueryDwelltime']= test['QueryDwelltime'].astype('float')
  test['Reformulationtime']= test['Reformulationtime'].astype('float')

In [ ]:
y_pred_serp = []
y_pred_query = []
y_pred_refor = []

for i in test.index:
  X = test[test.index==i]
  X_prep_serp = X.drop(["SERPtime", 'SERPDwelltime', 'QueryDwelltime', 'Reformulationtime'], axis=1)
  X_prep_serp  = X_prep_serp.to_numpy()
  X_prep_query = X.drop(["QueryDwellTime", 'QueryDwelltime', 'SERPDwelltime', 'Reformulationtime'], axis=1).to_numpy()
  X_prep_refor = X.drop(["ReformulationTime", 'QueryDwelltime', 'SERPDwelltime', 'Reformulationtime'], axis=1).to_numpy()
  serp = serp_rf_pretrained.predict(X_prep_serp)
  query = query_rf_pretrained.predict(X_prep_query)
  refor = refor_rf_pretrained.predict(X_prep_refor)
  y_pred_serp.append(serp)
  y_pred_query.append(query)
  y_pred_refor.append(refor)

In [ ]:
data['SERPDwelltime'] = serp_rf_pretrained.predict(prep_serp)
data['QueryDwelltime'] = query_rf_pretrained.predict(prep_query)
data['Reformulationtime'] = query_rf_pretrained.predict(prep_query)

In [ ]:
data.to_csv("input/file_8_all_qsegment_all.csv", sep = '\t', index=False, header=True)

# Step 9: Precdict Task States

To predict task states for each query statement, we used Kmeans clustering technique to classify each query segments into four cluster - explotation, explanatory, known-item, and learn/evaluate states.

In [8]:
data = pd.read_csv('input/file_9_all_qsegment.csv', sep = '\t')

In [10]:
data2 = pd.read_csv('input/file_7_first_qsegment_all_features.csv', sep = '\t')

In [ ]:
final = pd.concat([data,data2],axis=0).ffill(0)

In [17]:
final = final.drop(['index', 'id', 'id_y'], axis =1)

In [21]:
final = final.drop(['clean_query', 'clean_url', 'list_doc', 'title', 
                    'clean_title', 'list_title'], axis =1)

In [ ]:
final.sort_values(by = ['topic_id', 'query_id'])

In [26]:
final['title_length'].fillna(final['title_length'].mean(), inplace=True)
final['tf_title'].fillna(final['tf_title'].mean(), inplace=True)
final['title_idf'].fillna(final['title_idf'].mean(), inplace=True)
final['tfidf_title'].fillna(final['tfidf_title'].mean(), inplace=True)
final['title_bm25'].fillna(final['title_bm25'].mean(), inplace=True)
final['title_jarowinkler_sim'].fillna(final['title_jarowinkler_sim'].mean(), inplace=True)
final['title_jac_sim'].fillna(final['title_jac_sim'].mean(), inplace=True)

In [39]:
df = final.copy()

In [40]:
df.set_index('query_id', inplace=True)

In [42]:
df = df.drop(['doc_url', 'query'], axis = 1)

In [28]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt

In [29]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV

In [ ]:
names = df.columns
indexes = df.index
sc = MinMaxScaler((0, 1))
df1 = sc.fit_transform(df)
df2 = pd.DataFrame(df1, columns=names, index=indexes)
df2.head()

In [ ]:
kmeans = KMeans()
ssd = []
K = range(1, 15)

for k in K:
    kmeans = KMeans(n_clusters=k).fit(df2)
    ssd.append(kmeans.inertia_)

ssd

plt.plot(K, ssd, "bx-")
plt.xlabel("Distance Residual Sums for K Values")
plt.title("Elbow Method for Optimum Number of Clusters")
plt.show()

kmeans = KMeans()
visu = KElbowVisualizer(kmeans, k=(2, 10))
visu.fit(df2)
visu.show()

In [51]:
# Elbow method has identified 4 clusters, and the task state classification we used here also proposed 4 clusters.
kmeans = KMeans(n_clusters = 4).fit(df2)
clusters = kmeans.labels_

In [ ]:
pd.DataFrame({"states": final.index, "task_states": clusters})
final["task_states"] = clusters
final.head()

In [ ]:
final['task_states'].value_counts()

#2    820
#3    780
#1    760
#0    711

In [ ]:
# feature distribution for each cluster
for i in final.columns:
  plt.figure(figsize = (35, 5))
  for j in range(1,5):
    plt.subplot(1,5,j+1)
    cluster = final[final['task_states'] == j]
    cluster[i].hist(bins = 20)
    plt.title('{}    \nCluster {} '.format(i,j))
  plt.show()

In [60]:
final.to_csv('output/synthetic_session.csv', sep ='\t', index = None)